<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/9e_Multi_Output_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -l -s https://github.com/cagBRT/timeSeries.git cloned-repo
%cd cloned-repo

# **Mutli-Output MLP Model**

With a multi-output model, each output series is handled by a model<br>


In [ ]:
from IPython.display import Image
Image("MultiOutputMLP.png" , width=640)

**Import data**

In [ ]:
from numpy import array 
from numpy import hstack 
from keras.models import Model 
from keras.layers import Input
from keras.layers import Dense

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps): 
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences)-1: 
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :] 
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

**Create the data**

In [ ]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))]) 
in_seq1 = in_seq1.reshape((len(in_seq1), 1)) 
in_seq2 = in_seq2.reshape((len(in_seq2), 1)) 
out_seq = out_seq.reshape((len(out_seq), 1))
dataset = hstack((in_seq1, in_seq2, out_seq))

In [ ]:
dataset

**Set the number of steps**

In [ ]:
n_steps = 3

**Split the sequences into data and labels**

In [ ]:
X, y = split_sequences(dataset, n_steps)
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

Three steps per input<br>
10,20,30 -> 40<br>
15,25,35 -> 45<br>
40 + 45 = 85


In [ ]:
for i in range(len(X)):
 print(X[i], y[i])

Training a multi-output model requires separate output arrays for each sample.

In [ ]:
# separate output
y1 = y[:, 0].reshape((y.shape[0], 1)) 
y2 = y[:, 1].reshape((y.shape[0], 1)) 
y3 = y[:, 2].reshape((y.shape[0], 1))

In [ ]:
print(y1)

# **Define the model**

In [ ]:
visible = Input(shape=(n_input,))
dense = Dense(100, activation='relu')(visible)

**Define the outputs**

In [ ]:
output1 = Dense(1)(dense)
output2 = Dense(1)(dense)
output3 = Dense(1)(dense)

In [ ]:
model = Model(inputs=visible, outputs=[output1, output2, output3]) 
model.compile(optimizer='adam', loss='mse')

**Train the model**




In [ ]:
model.fit(X, [y1,y2,y3], epochs=2000, verbose=0)

In [ ]:
model.summary()

**Make Predictions**

In [ ]:
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_input))
print(x_input)

In [ ]:
yhat = model.predict(x_input, verbose=0)
print(yhat)

Expecting<br>
70,80,90 -> 100<br>
75,85,95 -> 105<br>
100 + 105 = 205<br>

In [ ]:
for i in range(len(X)):
 print(X[i], y[i])
print(x_input, yhat)

**Assignment:**<br>
1. Try different epoch values. What happens to yhat?
2. Add more data to the datset. Does the yhat improve?
3. Try going out to several time steps to make predictions.